In [3]:
# Dependencias Generales
import os
import gc
import nibabel as nib
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Dependencias de PyTorch
import torch
import torch.nn as nn
from torch.utils.data import random_split

In [5]:
# Fijar la semilla de PyTorch
torch.manual_seed(42)

# Fijar la semilla de NumPy
np.random.seed(42)

# Preprocesado

In [6]:
def arrayPaths(directory = '/kaggle/input'):
  """
   Recorrer recursivamente el directorio actual guardando los paths en un array.
   Ignorar metastasis.

  """
  paths = []

  # Recorre el directorio y sus subdirectorios, solo nivel superior
  for root, dirs, files in os.walk(directory):

      # Recorre las carpetas de primer nivel
      for subdir in dirs:
          subdir_path = os.path.join(root, subdir)
          seg_path = ""
          file_paths = []
          some_empty = False

          # Recorre los archivos dentro de las carpetas de primer nivel
          for file in os.listdir(subdir_path):
              file_path = os.path.join(subdir_path, file)
            
              if os.path.getsize(file_path) < 1: # Evitar archivos corruptos
                  some_empty = True
              if "seg" in file:  # Archivo de segmentación
                  seg_path = file_path
              else:
                  file_paths.append(file_path)

          # Agregar las rutas de archivos al array principal
          if "nii" in seg_path and not some_empty:
            paths.append((sorted(file_paths), seg_path))

  return paths

In [7]:
mri_paths = arrayPaths()

## Dividiendo en conjuntos de test, entrenamiento y validación

En este apartado dividimos el array global de ejemplos en datos de prueba (entrenamiento + validación) y test utilizando un esquema de $70\%$ y $30\%$. A su vez dividimos los datos de prueba en entrenamiento y validación siguiendo el mismo esquema.

In [8]:
test_size = int(0.3 * len(mri_paths))
mri_prueba, mri_test = random_split(mri_paths, [len(mri_paths)-test_size, test_size])

valid_size = int(0.3 * len(mri_prueba))
mri_train, mri_valid = random_split(mri_prueba, [len(mri_prueba)-valid_size, valid_size])

print(" -- Tamaño de los conjuntos -- ")
print('TEST: ', len(mri_test))
print('VALIDACIÓN: ', len(mri_valid))
print('TRAIN: ', len(mri_train))

 -- Tamaño de los conjuntos -- 
TEST:  632
VALIDACIÓN:  442
TRAIN:  1033


### Función para generar el CSV de preprocesado

In [7]:
def preprocessing(mri_paths, name='datasetAtlas.csv'): 
    """
    Genera un .csv que define el dataset con columnas (ruta_t1c, ruta_t1n, ruta_t2f, ruta_t2w, slice, etiqueta) 
    Etiquetas: 0 MENINGIOMA 1 GLIOMA 2 NO TUMOR
    Contiene todos los slices que tengan un tumor y una parte balanceada aleatoria de slices con NO TUMOR.
    
    """
    
    data = []
    
    print('Añadiendo los tumores en ', name)
    for mri_path in mri_paths:
        label_path = mri_path[1]
        
        # Iterar sobre la resonancia
        label_img = nib.load(label_path).get_fdata()
        
        for slice_num in range(5, 150):
            if np.any(label_img[:, :, slice_num] > 0):
                if 'MEN' in label_path:
                    label = 0
                else:
                    label = 1

                data.append({
                            "ruta_t1c": mri_path[0][0],  # Ruta al archivo T1c
                            "ruta_t1n": mri_path[0][1],  # Ruta al archivo T1n
                            "ruta_t2f": mri_path[0][2],  # Ruta al archivo T2f
                            "ruta_t2w": mri_path[0][3],  # Ruta al archivo T2w
                            "slice": slice_num,
                            "etiqueta": label
                        })
        del label_img
        gc.collect()
    
    print('Añadiendo los no tumores en ', name)
    notumores_size = len(data) // 2
    for i in range(0, notumores_size):
        
        # Valores aleatorios y comprobar que no es un tumor    
        slice_num = random.randint(5, 150)
        idx = random.randint(0, len(mri_paths) - 1)
    
        # Comprobar si el slice y los paths ya están en data
        if any(d["ruta_t1c"] == mri_paths[idx][0][0] and
               d["ruta_t1n"] == mri_paths[idx][0][1] and
               d["ruta_t2f"] == mri_paths[idx][0][2] and
               d["ruta_t2w"] == mri_paths[idx][0][3] and
               d["slice"] == slice_num
               for d in data):
            continue  # Si ya existe, continuar con la próxima iteración

        data.append({
            "ruta_t1c": mri_paths[idx][0][0],
            "ruta_t1n": mri_paths[idx][0][1],
            "ruta_t2f": mri_paths[idx][0][2],
            "ruta_t2w": mri_paths[idx][0][3],
            "slice": slice_num,
            "etiqueta": 2
        })
        
    # Convertir la lista de diccionarios a DataFrame
    df = pd.DataFrame(data)
    
    # Guardar el DataFrame como un archivo CSV
    df.to_csv(name, index=False)

In [8]:
# Preprocesar todos los conjuntos de CSV
preprocessing(mri_train, 'trainAtlas.csv')
preprocessing(mri_valid, 'validAtlas.csv')
preprocessing(mri_test, 'testAtlas.csv')

Añadiendo los tumores en  trainAtlas.csv
Añadiendo los no tumores en  trainAtlas.csv
Añadiendo los tumores en  validAtlas.csv
Añadiendo los no tumores en  validAtlas.csv
Añadiendo los tumores en  testAtlas.csv
Añadiendo los no tumores en  testAtlas.csv
